In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(1)

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

sns.set_palette("Spectral")

# Set a base style
sns.set_style("whitegrid")

# Customize specific style parameters
custom_params = {
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.color": "#cccccc",
    "axes.facecolor": "#f0f0f0",
    "axes.labelsize": 14,
    "axes.titlesize": 16,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12
}
sns.set_context("notebook", rc=custom_params)

In [1]:
import numpy as np
from FizNet import NN

In [2]:
layer_dims = [2, 3, 2, 2]  # 2 input features, 2 hidden layers (3 and 2 neurons), 2 output classes

nn = NN(layer_dims)

# Input features X (2 features, 3 samples)
X = np.array([
    [1, 2, 3],
    [0.1, 0.2, 0.3]
])

# Target labels Y (2 classes, 3 samples)
Y = np.array([
    [1, 0, 1],
    [0, 1, 0]
])

print("X shape:", X.shape)
print("Y shape:", Y.shape)

nn.train(X, Y, num_iterations=1, learning_rate=0.1)


Layer 1: W1 shape: (3, 2), b1 shape: (3, 1)
Layer 2: W2 shape: (2, 3), b2 shape: (2, 1)
Layer 3: W3 shape: (2, 2), b3 shape: (2, 1)
Parameters initialized: dict_keys(['W1', 'b1', 'W2', 'b2', 'W3', 'b3'])
Parameters; {'W1': array([[ 1.62434536, -0.61175641],
       [-0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ]]), 'b1': array([[0.],
       [0.],
       [0.]]), 'W2': array([[ 1.42463284, -0.62152283,  0.26049433],
       [-0.20361006,  1.19380613, -1.68209785]]), 'b2': array([[0.],
       [0.]]), 'W3': array([[-0.3224172 , -0.38405435],
       [ 1.13376944, -1.09989127]]), 'b3': array([[0.],
       [0.]])}
Number of layers: 3
X shape: (2, 3)
Y shape: (2, 3)
cahces: []
A: [[1.  2.  3. ]
 [0.1 0.2 0.3]]
Layer 1
A_prev: [[1.  2.  3. ]
 [0.1 0.2 0.3]]
A_prev shape: (2, 3), W shape: (3, 2), b shape: (3, 1)
A_prev: [[1.  2.  3. ]
 [0.1 0.2 0.3]], W: [[ 1.62434536 -0.61175641]
 [-0.52817175 -1.07296862]
 [ 0.86540763 -2.3015387 ]], b: [[0.]
 [0.]
 [0.]]
A: [[1.56316972 3.12633944

[4.655369989120831]

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


# Load the digits dataset
digits = load_digits()
X, y = digits.data, digits.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data to (n_features, n_samples) as required by the NN
X_train = X_train.T
X_test = X_test.T

# One-hot encode the labels
def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y].T

y_train_one_hot = one_hot_encode(y_train, 10)
y_test_one_hot = one_hot_encode(y_test, 10)

# Define neural network architecture
input_size = X_train.shape[0]
hidden_layer1_size = 128
hidden_layer2_size = 64
num_classes = 10
layer_dims = [input_size, hidden_layer1_size, hidden_layer2_size, num_classes]

nn = NN(layer_dims)

# Perform cross-validation
kf = KFold(n_splits=5)
fold = 1
for train_index, val_index in kf.split(X_train.T):
    X_train_fold, X_val_fold = X_train[:, train_index], X_train[:, val_index]
    y_train_fold, y_val_fold = y_train_one_hot[:, train_index], y_train_one_hot[:, val_index]
    
    # Train the model
    costs = nn.train(X_train_fold, y_train_fold, num_iterations=2500, learning_rate=0.1)
    
    # Make predictions for the validation fold
    val_predictions = nn.predict(X_val_fold)
    
    # Calculate accuracy for this fold
    val_accuracy = accuracy_score(np.argmax(y_val_fold, axis=0), val_predictions)
    print(f"Fold {fold} Validation Accuracy: {val_accuracy:.4f}")
    fold += 1

# Final training on the entire training set
costs = nn.train(X_train, y_train_one_hot, num_iterations=2500, learning_rate=0.1)

# Make predictions for the test set
test_predictions = nn.predict(X_test)

# Calculate accuracy
test_accuracy = accuracy_score(np.argmax(y_test_one_hot, axis=0), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(np.argmax(y_test_one_hot, axis=0), test_predictions))

# Plot the cost over iterations
plt.figure(figsize=(8, 6))
sns.lineplot(x=range(0, 2500, 100), y=costs, marker='o', color='#e74645', linewidth=2)
plt.title('Cost vs. Iterations')
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

# Visualize some predictions
fig, axes = plt.subplots(2, 5, figsize=(8, 4))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_test[:, i].reshape(8, 8), cmap='gray')
    ax.set_title(f"Pred: {test_predictions[i]}, True: {np.argmax(y_test_one_hot[:, i])}")
    ax.axis('off')
plt.tight_layout()
plt.show()